<a href="https://colab.research.google.com/github/mttbanizi/Bearing_condition_machine-learning/blob/main/Bearing_condition_machine_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import seaborn as sns
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import scipy.io # To use the '.mat' files


In [5]:
path = "/content/drive/MyDrive/0_load_48_KHz/IR007_0.mat"
mat = scipy.io.loadmat(path)

In [4]:
for root, dirs, files in os.walk("/content/drive/MyDrive/0_load_48_KHz", topdown=False):
    for file_name in files:
        path = os.path.join(root, file_name)
        print(path)

/content/drive/MyDrive/0_load_48_KHz/IR007_0.mat
/content/drive/MyDrive/0_load_48_KHz/B007_0.mat
/content/drive/MyDrive/0_load_48_KHz/OR007@3_0.mat
/content/drive/MyDrive/0_load_48_KHz/OR007@6_0.mat
/content/drive/MyDrive/0_load_48_KHz/OR007@12_0.mat


In [6]:
mat.items()

dict_items([('__header__', b'MATLAB 5.0 MAT-file, Platform: PCWIN, Created on: Mon Jan 31 15:28:48 2000'), ('__version__', '1.0'), ('__globals__', []), ('X109_DE_time', array([[ 0.010016  ],
       [-0.023788  ],
       [-0.00792933],
       ...,
       [-0.085136  ],
       [-0.05800933],
       [ 0.00542533]])), ('X109_FE_time', array([[-0.35757879],
       [-0.24455101],
       [-0.1870096 ],
       ...,
       [-1.17137879],
       [-1.11630343],
       [-0.98930131]])), ('X109RPM', array([[1796]], dtype=uint16))])

In [7]:
list(mat.keys())

['__header__',
 '__version__',
 '__globals__',
 'X109_DE_time',
 'X109_FE_time',
 'X109RPM']

In [8]:
key_name = list(mat.keys())[3]
key_name

'X109_DE_time'

In [9]:
DE_data = mat.get(key_name)
DE_data

array([[ 0.010016  ],
       [-0.023788  ],
       [-0.00792933],
       ...,
       [-0.085136  ],
       [-0.05800933],
       [ 0.00542533]])

In [11]:
fault = np.full((len(DE_data), 1), file_name[:-4])

In [12]:
df_temp = pd.DataFrame({'DE_data':np.ravel(DE_data) , 'fault':np.ravel(fault)})


In [13]:
df=pd.DataFrame(columns=['DE_data','fault'])

for root, dirs, files in os.walk("/content/drive/MyDrive/0_load_48_KHz", topdown=False):
    for file_name in files:
        path = os.path.join(root, file_name)
        print(path)

        mat = scipy.io.loadmat(path)

        key_name = list(mat.keys())[3]
        DE_data = mat.get(key_name)
        fault = np.full((len(DE_data), 1), file_name[:-4])

        df_temp = pd.DataFrame({'DE_data':np.ravel(DE_data) , 'fault':np.ravel(fault)})

        df = pd.concat([df,df_temp],axis=0)
        print(df['fault'].unique())

df.to_csv('all_faults.csv',index=False)

/content/drive/MyDrive/0_load_48_KHz/IR007_0.mat
['IR007_0']
/content/drive/MyDrive/0_load_48_KHz/B007_0.mat
['IR007_0' 'B007_0']
/content/drive/MyDrive/0_load_48_KHz/OR007@3_0.mat
['IR007_0' 'B007_0' 'OR007@3_0']
/content/drive/MyDrive/0_load_48_KHz/OR007@6_0.mat
['IR007_0' 'B007_0' 'OR007@3_0' 'OR007@6_0']
/content/drive/MyDrive/0_load_48_KHz/OR007@12_0.mat
['IR007_0' 'B007_0' 'OR007@3_0' 'OR007@6_0' 'OR007@12_0']


In [14]:
df.to_csv('/content/drive/MyDrive/0_load_48_KHz/all_faults.csv',index=False)

In [15]:
df = pd.read_csv('/content/drive/MyDrive/0_load_48_KHz/all_faults.csv')


In [16]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

win_len=1000
stride=200

X=[]
Y=[]


for k in df['fault'].unique():

    df_temp_2 = df[df['fault']==k]

    for i in np.arange(0,len(df_temp_2)-(win_len),stride):
        temp = df_temp_2.iloc[i:i+win_len,:-1].values
        temp = temp.reshape((1,-1))
        X.append(temp)
        Y.append(df_temp_2.iloc[i+win_len,-1])

X=np.array(X)
X=X.reshape((X.shape[0],win_len))



Y=np.array(Y)
encoder= LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
OHE_Y = to_categorical(encoded_Y)

In [ ]:
l.var()

tensor(0.0852)

In [ ]:
5+2

7

In [ ]:
l[1:, -1 , :]

tensor([[0.2583, 0.6084, 0.6042, 0.6416, 0.4229],
        [0.4173, 0.7210, 0.3459, 0.2429, 0.8881]])

In [ ]:
l[:,:,-1]

tensor([[0.2405, 0.5932, 0.5799, 0.9339],
        [0.3124, 0.8340, 0.9623, 0.6151],
        [0.8195, 0.7295, 0.2670, 0.8590]])

In [ ]:
l[-2,:,:]

tensor([[0.2161, 0.8613, 0.1472, 0.2214, 0.3124],
        [0.9060, 0.8641, 0.1854, 0.4803, 0.8340],
        [0.8982, 0.1762, 0.1969, 0.8939, 0.9623],
        [0.1628, 0.5846, 0.4998, 0.6003, 0.6151]])

In [ ]:
l[1,2]

tensor([0.8982, 0.1762, 0.1969, 0.8939, 0.9623])

In [ ]:
  l[1,1]

tensor([0.2052, 0.0856, 0.4079, 0.8893, 0.2771])

In [ ]:
l

tensor([[[0.8024, 0.4830, 0.1244, 0.0623, 0.4106],
         [0.7999, 0.1068, 0.6359, 0.8178, 0.0772],
         [0.7675, 0.0701, 0.7380, 0.0139, 0.7192],
         [0.0712, 0.6802, 0.7633, 0.3418, 0.2037]],

        [[0.9713, 0.9200, 0.1783, 0.7675, 0.7740],
         [0.2052, 0.0856, 0.4079, 0.8893, 0.2771],
         [0.5076, 0.7150, 0.9338, 0.4961, 0.5271],
         [0.2321, 0.2839, 0.7435, 0.1664, 0.0639]],

        [[0.1146, 0.1472, 0.8061, 0.5035, 0.3159],
         [0.8026, 0.6327, 0.3144, 0.0263, 0.1568],
         [0.8505, 0.7732, 0.6105, 0.3535, 0.5210],
         [0.5527, 0.2040, 0.6038, 0.2769, 0.2897]]])